In [6]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("A {word} is a")

llm = HuggingFacePipeline.from_model_id(
    model_id="gpt2",
    task="text-generation",
    pipeline_kwargs={"max_new_tokens": 150},
)

chain = prompt | llm

chain.invoke({"word": "potato"})

Using pad_token, but it is not set yet.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


' small, very large potato. When first mashed, the potato is hard and slightly sticky, the potato is hard, a little sticky when cold, and the potato is harder than it is warm. The texture of the potato mixture changes from brown to hard and is very hard when frozen. If you have a potato masher, a potato masher or a potato masher mixer, you can make your potato masher in half length, and half circumference ratio (half potato or 1/2 potato).\n\nStep 5: Heat a large skillet or pan on medium heat. Add the onion, garlic, shallot, thyme, tomato, lemon grated, and salt. Cook for 5-7 minutes to remove. Add any chopped fresh greens'